In [7]:
#!/usr/bin/env python3
# pypylon_with_preview_and_resize.py

import cv2
import datetime
import os
from pypylon import pylon, genicam

def record_with_pfs_and_preview(output_filepath: str,
                                parameter_filepath: str,
                                record_seconds: int = 10,
                                override_settings: dict = None,
                                output_size=(640, 480)):
    """
    使用 PyPylon 从 Basler 相机加载 PFS 参数文件，录制视频并实时预览，预览窗口叠加计时，保存视频不带文字，支持自定义分辨率。

    参数:
        output_filepath: 输出 AVI 文件的完整路径
        parameter_filepath: 相机参数文件 (.pfs/.dat) 的路径
        record_seconds: 录制时长（秒）
        override_settings: 可选，字典形式覆盖 PFS 中的设置（例如 {'ExposureTime': 10000}）
        output_size: 输出视频分辨率（宽, 高）
    """
    # 确保输出目录存在
    out_dir = os.path.dirname(output_filepath)
    if out_dir and not os.path.exists(out_dir):
        os.makedirs(out_dir, exist_ok=True)

    # 初始化相机
    tl_factory = pylon.TlFactory.GetInstance()
    devices = tl_factory.EnumerateDevices()
    if not devices:
        raise RuntimeError("未检测到任何 Basler 相机")
    camera = pylon.InstantCamera(tl_factory.CreateDevice(devices[0]))
    camera.Open()

    # 加载 PFS 参数
    try:
        pylon.FeaturePersistence.Load(parameter_filepath, camera.GetNodeMap(), True)
        print(f"已加载参数文件: {parameter_filepath}")
    except genicam.GenericException as e:
        raise RuntimeError(f"加载参数文件失败: {e}")

    # 可选：覆盖部分设置
    if override_settings:
        for name, value in override_settings.items():
            try:
                getattr(camera, name).SetValue(value)
                print(f"覆盖参数: {name} = {value}")
            except Exception:
                print(f"警告：无法覆盖参数 {name}")

    width = camera.Width.GetValue()
    height = camera.Height.GetValue()
    fps_enable = camera.AcquisitionFrameRateEnable.GetValue()
    fps = camera.AcquisitionFrameRate.GetValue() if fps_enable else 30
    pix_fmt = camera.PixelFormat.GetValue()
    is_color = 'Bayer' in pix_fmt or 'RGB' in pix_fmt or 'BGR' in pix_fmt
    print(f"相机设置: 分辨率={width}x{height}, 帧率={fps}fps, 像素格式={pix_fmt}")

    camera.StartGrabbing(pylon.GrabStrategy_LatestImageOnly)
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    video_writer = cv2.VideoWriter(output_filepath, fourcc, fps, output_size, isColor=is_color)
    print(f"开始录制: {output_filepath}，时长: {record_seconds}s，输出分辨率: {output_size}")

    start_time = datetime.datetime.now()
    frame_count = 0
    while (datetime.datetime.now() - start_time).total_seconds() < record_seconds:
        grab_result = camera.RetrieveResult(5000, pylon.TimeoutHandling_ThrowException)
        if grab_result.GrabSucceeded():
            img = grab_result.Array
            if is_color and img.ndim == 3:
                if 'RGB' in pix_fmt:
                    frame = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
                elif 'Bayer' in pix_fmt:
                    frame = cv2.cvtColor(img, cv2.COLOR_BAYER_BG2BGR)
                else:
                    frame = img
            else:
                frame = img

            # 缩放用于保存的视频帧
            resized_frame = cv2.resize(frame, output_size, interpolation=cv2.INTER_AREA)
            video_writer.write(resized_frame)  # 保存时不加文字

            # 只在预览窗口显示计时
            preview_frame = resized_frame.copy()
            elapsed_time = (datetime.datetime.now() - start_time).total_seconds()
            elapsed_str = f"{int(elapsed_time // 60):02d}:{int(elapsed_time % 60):02d}"
            cv2.putText(preview_frame, f"Time: {elapsed_str}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
            cv2.imshow("Live Video", preview_frame)

            if cv2.waitKey(1) & 0xFF == 27:  # 按 ESC 键提前退出
                print("录制提前终止")
                break

            frame_count += 1
            if frame_count % 100 == 0:
                print(f"已录制帧数: {frame_count}")
        grab_result.Release()

    camera.StopGrabbing()
    camera.Close()
    video_writer.release()
    cv2.destroyAllWindows()
    print("录制完成并保存文件")

if __name__ == "__main__":
    # 输出文件路径（示例）
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    animal = "67606"
    day = "day5"
    mode = "test_"
    #mode=None
    output_file = fr"S:\Sachuriga\Ephys_Vedio\CRs_memory\{day}\{animal}_{day}_{mode}{timestamp}.avi"
    # PFS 参数文件路径
    parameter_file = fr"Q:\sachuriga\Record_archive\Bonsai_settings\OpenFiledRecoding.pfs"

    try:
        record_with_pfs_and_preview(output_file,
                                    parameter_file,
                                    record_seconds=600,
                                    override_settings=None,
                                    output_size=(850, 850))  # 可自定义分辨率
    except Exception as e:
        print(f"录制失败: {e}")


已加载参数文件: Q:\sachuriga\Record_archive\Bonsai_settings\OpenFiledRecoding.pfs
相机设置: 分辨率=880x880, 帧率=30.0fps, 像素格式=Mono8
开始录制: S:\Sachuriga\Ephys_Vedio\CRs_memory\day5\67606_day5_test_20250710_172656.avi，时长: 600s，输出分辨率: (850, 850)
已录制帧数: 100
已录制帧数: 200
已录制帧数: 300
已录制帧数: 400
已录制帧数: 500
已录制帧数: 600
已录制帧数: 700
已录制帧数: 800
已录制帧数: 900
已录制帧数: 1000
已录制帧数: 1100
已录制帧数: 1200
已录制帧数: 1300
已录制帧数: 1400
已录制帧数: 1500
已录制帧数: 1600
已录制帧数: 1700
已录制帧数: 1800
已录制帧数: 1900
已录制帧数: 2000
已录制帧数: 2100
已录制帧数: 2200
已录制帧数: 2300
已录制帧数: 2400
已录制帧数: 2500
已录制帧数: 2600
已录制帧数: 2700
已录制帧数: 2800
已录制帧数: 2900
已录制帧数: 3000
已录制帧数: 3100
已录制帧数: 3200
已录制帧数: 3300
已录制帧数: 3400
已录制帧数: 3500
已录制帧数: 3600
已录制帧数: 3700
已录制帧数: 3800
已录制帧数: 3900
已录制帧数: 4000
已录制帧数: 4100
已录制帧数: 4200
已录制帧数: 4300
已录制帧数: 4400
已录制帧数: 4500
已录制帧数: 4600
已录制帧数: 4700
已录制帧数: 4800
已录制帧数: 4900
已录制帧数: 5000
已录制帧数: 5100
已录制帧数: 5200
已录制帧数: 5300
已录制帧数: 5400
已录制帧数: 5500
已录制帧数: 5600
已录制帧数: 5700
已录制帧数: 5800
已录制帧数: 5900
已录制帧数: 6000
已录制帧数: 6100
已录制帧数: 6200
已录制帧数: 6300
已录制帧数: 6400
已录制帧数: 6500
已录